In [1]:
import pandas as pd
from models import scalers, decompositions, estimators, create_params_grid
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from imblearn.metrics import classification_report_imbalanced
from imblearn.pipeline import Pipeline
import pandas as pd
import neptune.new as neptune

In [39]:
run = neptune.init(project='projektmlcdv/ml-project-MW-MM', api_token = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIwYmI4NjVmYS0zM2JlLTRkZTQtOTI5YS0zNjY4NzhjNGQ4YjYifQ==')

https://app.neptune.ai/projektmlcdv/ml-project-MW-MM/e/MLPROJ-36
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [3]:
df_reg = pd.read_html('cv_results_reg.html')

In [4]:
df_svc = pd.read_html('cv_results_svc.html')

In [5]:
df_tree = pd.read_html('cv_results_tree.html')

In [6]:
pd.set_option("display.max_columns", 101)

In [7]:
df_tree[0].sort_values(by=['rank_test_f1_weighted', 'rank_test_recall_weighted'],inplace = True)

In [8]:
df_reg[0].sort_values(by=['rank_test_f1_weighted', 'rank_test_recall_weighted'], inplace = True)

In [9]:
df_svc[0].sort_values(by=['rank_test_f1_weighted', 'rank_test_recall_weighted'],inplace = True)

In [10]:
df_reg[0]['param_estimator__solver'].unique()

array(['liblinear', 'sag', 'newton-cg', 'lbfgs', 'saga'], dtype=object)

#### Params

Tree

In [12]:
list(df_tree[0].head(1).params)

["{'decomposition': PCA(), 'decomposition__n_components': 2, 'decomposition__svd_solver': 'full', 'decomposition__whiten': False, 'estimator': DecisionTreeClassifier(), 'estimator__class_weight': 'balanced', 'estimator__criterion': 'entropy', 'estimator__max_depth': 2, 'estimator__splitter': 'random', 'scaler': StandardScaler()}"]

Reg

In [13]:
list(df_reg[0].head(1).params)

["{'decomposition': KernelPCA(), 'decomposition__gamma': 0.03, 'decomposition__kernel': 'linear', 'estimator': LogisticRegression(), 'estimator__C': 1.0, 'estimator__class_weight': 'balanced', 'estimator__penalty': 'l1', 'estimator__solver': 'liblinear', 'scaler': StandardScaler()}"]

SVC

In [14]:
list(df_svc[0].head(1).params)

["{'decomposition': PCA(), 'decomposition__n_components': 10, 'decomposition__svd_solver': 'auto', 'decomposition__whiten': False, 'estimator': SVC(), 'estimator__C': 35.938136638046274, 'estimator__class_weight': 'balanced', 'estimator__gamma': 'auto', 'estimator__kernel': 'rbf', 'scaler': StandardScaler()}"]

Params grid

In [15]:
params_grid = [{'scaler': [StandardScaler()],
 'decomposition': [PCA()],
 'decomposition__n_components': [2],
 'decomposition__whiten': [False],
 'decomposition__svd_solver': ['full'],
 'estimator': [DecisionTreeClassifier()],
 'estimator__class_weight': ['balanced'],
 'estimator__criterion': ['entropy'],
 'estimator__max_depth': [2],
 'estimator__splitter': ['random']},
               
               

              {'scaler': [StandardScaler()],
 'decomposition': [KernelPCA()],
 'decomposition__gamma': [0.03],
 'decomposition__kernel': ['linear'], 
 'estimator': [LogisticRegression()], 
 'estimator__C': [1.0], 
 'estimator__class_weight': ['balanced'], 
 'estimator__penalty': ['l1'], 
 'estimator__solver': ['liblinear']},
               
        
               
    {'scaler': [StandardScaler()],
    'decomposition': [PCA()],
    'decomposition__n_components': [10], 
    'decomposition__svd_solver': ['auto'], 
    'decomposition__whiten': [False], 
    'estimator': [SVC()], 
    'estimator__C': [35.938136638046274], 
    'estimator__class_weight': ['balanced'], 
    'estimator__gamma':[ 'auto'],
    'estimator__kernel': ['rbf']}]

In [30]:
pipe = Pipeline(steps=[('scaler', StandardScaler()),('decomposition',PCA()),('estimator', SVC())])

In [31]:
encoder = lambda x:1 if x == -1 else 0
decoder = lambda x:-1 if x == 1 else 1

In [32]:
y = pd.read_csv('train_labels.csv', header=None, names=['y'])
X = pd.read_csv("train_data.csv",header=None, low_memory = False)

In [33]:
y= y['y'].apply(encoder)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,random_state =40)

In [38]:
grid = GridSearchCV(pipe, params_grid,verbose = 2, cv=4, n_jobs = -1, scoring = 'f1_weighted',refit = True,)

In [40]:
best = grid.fit(X_train,y_train)
run["cv_results"].upload(neptune.types.File.as_html(pd.DataFrame.from_dict(best.cv_results_)))

Fitting 4 folds for each of 3 candidates, totalling 12 fits


In [ ]:
best.